In [8]:
import os
import shutil
import yaml
import json
import PIL
import ultralytics
import numpy as np

from pathlib import Path
from ultralytics.data.converter import convert_coco

# Conversion

In [9]:
# Convert filenames in COCO
rel_coco_dir = "data/COCO_original"
coco_labels_path = f"{rel_coco_dir}/result.json"
coco_labels_path_filtered = f"data/COCO/result.json"
img_reldir = f"images"
with open(coco_labels_path, "r") as f:
    coco_data = json.load(f)
for image in coco_data["images"]:
    path = Path(image["file_name"])
    filename = path.name
    image["file_name"] = str(Path(img_reldir) / filename)

with open(coco_labels_path_filtered, "w") as f:
    json.dump(coco_data, f, indent=4)


In [10]:
# Make train test split
train_coco = {"images": [], "annotations": [], "categories": coco_data["categories"], "info": coco_data["info"]}
test_coco = {"images": [], "annotations": [], "categories": coco_data["categories"], "info": coco_data["info"]}

img_ids = [img["id"] for img in coco_data["images"]]
img_ids_shuffled = np.random.permutation(img_ids)
TRAIN_PORTION = 0.9
train_size = int(TRAIN_PORTION * len(img_ids))
train_ids = img_ids_shuffled[:train_size]
test_ids = img_ids_shuffled[train_size:]

for image in coco_data["images"]:
    if image["id"] in train_ids:
        train_coco["images"].append(image)
    else:
        test_coco["images"].append(image)

for annot in coco_data["annotations"]:
    if annot["image_id"] in train_ids:
        train_coco["annotations"].append(annot)
    else:
        test_coco["annotations"].append(annot)

In [11]:
for image in train_coco["images"]:
    filename = Path(image["file_name"])
    

# YOLO Train Val Split

In [12]:
# Create temp dir
try:
    shutil.rmtree("data/ATPgesamt/temp")
except FileNotFoundError:
    pass
os.makedirs("data/ATPgesamt/temp/images/train", exist_ok=True)
os.makedirs("data/ATPgesamt/temp/images/val", exist_ok=True)
os.makedirs("data/ATPgesamt/temp/labels/train", exist_ok=True)
os.makedirs("data/ATPgesamt/temp/labels/val", exist_ok=True)
data_dir = "data/ATPgesamt"

In [13]:
# Get all filenames
image_filepaths = []
with open(f"{data_dir}/train.txt", "r") as file:
    for line in file:
        image_filepath = line.strip()
        image_filepath = str(Path(data_dir) / Path(*Path(image_filepath).parts[1:]))
        image_filepaths.append(image_filepath)
image_filenames = [Path(fp).name for fp in image_filepaths]
image_filenames_name = [Path(fp).stem for fp in image_filepaths]
label_filenames = [f"{name}.txt" for name in image_filenames_name]
label_filepaths = [str(Path(data_dir) / "labels" / "train" / lf) for lf in label_filenames]

# Unlabeled images do not have label files
labeled_image_filepaths = []
labeled_label_filepaths = []
for img_fp, lbl_fp in zip(image_filepaths, label_filepaths):
    if os.path.exists(lbl_fp):
        labeled_image_filepaths.append(img_fp)
        labeled_label_filepaths.append(lbl_fp)
assert len(labeled_image_filepaths) == len(os.listdir(f"{data_dir}/labels/train"))
# Take only labeled files
label_filepaths = labeled_label_filepaths
image_filepaths = labeled_image_filepaths

In [14]:
# Choose random train/val split

VAL_PORTION = 0.1
num_images = len(image_filepaths)
num_val = int(VAL_PORTION * num_images)
shuffled_indices = np.random.permutation(num_images)
val_indices = shuffled_indices[:num_val]
train_indices = shuffled_indices[num_val:]
# Move files to temp dir
for idx in val_indices:
    img_fp = image_filepaths[idx]
    lbl_fp = label_filepaths[idx]
    assert os.path.exists(img_fp), f"Image file does not exist: {img_fp}"
    assert os.path.exists(lbl_fp), f"Label file does not exist: {lbl_fp}"
    shutil.copy2(img_fp, f"{data_dir}/temp/images/val/{Path(img_fp).name}")
    shutil.copy2(lbl_fp, f"{data_dir}/temp/labels/val/{Path(lbl_fp).name}")
for idx in train_indices:
    img_fp = image_filepaths[idx]
    lbl_fp = label_filepaths[idx]
    assert os.path.exists(img_fp), f"Image file does not exist: {img_fp}"
    assert os.path.exists(lbl_fp), f"Label file does not exist: {lbl_fp}"
    shutil.copy2(img_fp, f"{data_dir}/temp/images/train/{Path(img_fp).name}")
    shutil.copy2(lbl_fp, f"{data_dir}/temp/labels/train/{Path(lbl_fp).name}")
# Create new train.txt and val.txt
with open(f"{data_dir}/temp/train.txt", "w") as train_file:
    for idx in train_indices:
        img_fp = image_filepaths[idx]
        train_file.write(f"{data_dir}/temp/images/train/{Path(img_fp).name}\n")
with open(f"{data_dir}/temp/val.txt", "w") as val_file:
    for idx in val_indices:
        img_fp = image_filepaths[idx]
        val_file.write(f"{data_dir}/temp/images/val/{Path(img_fp).name}\n")
# Create new data.yaml
data_yaml = {
    "train": f"train.txt",
    "val": f"val.txt",
    "path": f"{data_dir}/temp",
    "names": {0: "Screw"}
}
with open(f"{data_dir}/temp/data.yaml", "w") as yaml_file:
    yaml.dump(data_yaml, yaml_file)

# Verify that training and val images are disjoint
train_images = set(os.listdir(f"{data_dir}/temp/images/train"))
val_images = set(os.listdir(f"{data_dir}/temp/images/val"))
assert train_images.isdisjoint(val_images), "Train and validation images are not disjoint!"

# Training

In [15]:
model = ultralytics.YOLO(model="yolo11n.pt")

# Make data augmentation heavier
model.train(
    data="data/ATPgesamt/temp/data.yaml",
    epochs=100,
    imgsz=1024,
    batch=16,
    augment=True,
    cutmix=0.1,
    bgr=0.1,
    perspective=0.0001,
    shear=10
)

New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3090, 24117MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.1, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.1, data=data/ATPgesamt/temp/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train10, nbs=64, nms

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x77c12f3c53d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

Open questions:
- Data augmentation? Colored, backgrounds?
- Underlying structure known? (e.g. max 5 or 5 in a certain polygon)
  - Each screw could be its own type, screw0-screw4 -> Does YOLO already take into account global structure?
- Maybe segment the part as well?

# Inference

In [2]:
model = ultralytics.YOLO("runs/detect/train8/weights/best.pt")

In [3]:
model.export(format="onnx")

Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.7.1+cu126 CPU (AMD Ryzen 5 7600X 6-Core Processor)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'runs/detect/train8/weights/best.pt' with input shape (1, 3, 768, 768) BCHW and output shape(s) (1, 5, 12096) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/18.2 MB ? eta -:--:--
   ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/18.2 MB 7.7 MB/s eta 0:00:03
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/18.2 MB 9.9 MB/s eta 0:00:02
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/18.2 MB 10.5 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━ 8.7/18.2 MB 10.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━ 10.7/18.2 MB 10.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 13.1/18.2 MB 11.1 MB/s eta 0:00:01
   ━

'runs/detect/train8/weights/best.onnx'

In [14]:
val_images = os.listdir("data/yolo224/temp/images/val")
val_image_paths = [f"data/yolo224/temp/images/val/{img_name}" for img_name in val_images]

In [16]:
images = []
for path in val_image_paths:
    images.append(PIL.Image.open(path))
assert len(images) == len(val_image_paths)
results = model.predict(images)
for result in results:
    result.show()


0: 768x768 2 Screws, 1.5ms
1: 768x768 6 Screws, 1.5ms
2: 768x768 5 Screws, 1.5ms
3: 768x768 3 Screws, 1.5ms
4: 768x768 2 Screws, 1.5ms
5: 768x768 5 Screws, 1.5ms
6: 768x768 3 Screws, 1.5ms
7: 768x768 5 Screws, 1.5ms
8: 768x768 5 Screws, 1.5ms
9: 768x768 5 Screws, 1.5ms
10: 768x768 3 Screws, 1.5ms
11: 768x768 4 Screws, 1.5ms
12: 768x768 5 Screws, 1.5ms
13: 768x768 4 Screws, 1.5ms
14: 768x768 5 Screws, 1.5ms
15: 768x768 1 Screw, 1.5ms
16: 768x768 2 Screws, 1.5ms
17: 768x768 5 Screws, 1.5ms
18: 768x768 5 Screws, 1.5ms
19: 768x768 5 Screws, 1.5ms
20: 768x768 5 Screws, 1.5ms
21: 768x768 3 Screws, 1.5ms
Speed: 3.6ms preprocess, 1.5ms inference, 0.3ms postprocess per image at shape (1, 3, 768, 768)
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Op

Opening in existing browser session.


In [18]:
model.val()

Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3090, 24090MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 7151.9±905.0 MB/s, size: 6416.9 KB)
val: Scanning data/yolo224/temp/labels/val.cache... 22 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 22/22 70.8Kit/s 0.0s
val: data/yolo224/temp/images/val/20251203_174842.jpg: corrupt JPEG restored and saved
val: data/yolo224/temp/images/val/20251203_174846.jpg: corrupt JPEG restored and saved
val: data/yolo224/temp/images/val/20251203_174852.jpg: corrupt JPEG restored and saved
val: data/yolo224/temp/images/val/20251203_174949.jpg: corrupt JPEG restored and saved
val: data/yolo224/temp/images/val/20251203_175904.jpg: corrupt JPEG restored and saved
val: data/yolo224/temp/images/val/20251203_175958.jpg: corrupt JPEG restored and saved
val: data/yolo224/temp/images/val/20251203_180002.jpg: corrupt JPEG restored and saved
val: data/yolo224/temp/images/val/20251203_180011.jpg: corrupt JPEG restored an

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aeeb82bea80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# Roboflow model

In [9]:
from inference_sdk import InferenceHTTPClient

client = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="4CMPVxpyWwGUWvP3yQFy"
)

result = client.run_workflow(
    workspace_name="atp-iad7y",
    workflow_id="find-screws",
    images={
        "image": "/home/marius-neuhalfen/Documents/Perso/RWTH/ATP/RoboSchrauber/data/COCO/images/c49774ff-WhatsApp_Image_2025-11-26_at_17.47.391.jpeg"
    },
    use_cache=True # cache workflow definition for 15 minutes
)
